In [1]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer




2025-06-03 23:25:37.711008: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-06-03 23:25:37.711223: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-06-03 23:25:37.838945: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-06-03 23:25:38.101448: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-06-03 23:25:41.147183: W tensorflow/compiler/tf2

In [ ]:
# ============================================================
# finetune_emotion_es.py
# Fine-tuning de “daveni/twitter-xlm-roberta-emotion-es”
# sobre tu dataset de tweets en español con etiquetas emocionales
# ============================================================

# 1) ──────────────── LIBRERÍAS ──────────────────────────────
from datasets   import Dataset
from transformers import (
    AutoTokenizer, AutoModelForSequenceClassification,
    TrainingArguments, Trainer
)
from sklearn.metrics import accuracy_score, f1_score
import pandas as pd, numpy as np, re, torch, pathlib, csv, os

# 2) ──────────────── CARGA + LIMPIEZA DEL CSV ───────────────
def detect_delimiter(path):
    """Detecta automáticamente ',', ';' o tab."""
    with open(path, "r", encoding="utf-8") as f:
        sample = "".join([f.readline() for _ in range(5)])
    return csv.Sniffer().sniff(sample, delimiters=[",", ";", "\t"]).delimiter

CSV_PATH = "dataset.csv"
delimiter = detect_delimiter(CSV_PATH)

def clean_tweet(text: str) -> str:
    """• Quita URLs, @menciones, hashtags y tokens HASHTAG/USER
       • Colapsa espacios múltiples."""
    text = re.sub(r"https?://\S+", " ", text)            # URLs
    text = re.sub(r"@\w+", " ", text)                    # @menciones reales
    text = re.sub(r"#\w+", " ", text)                    # hashtags reales (#libros)
    text = re.sub(r"\b(HASHTAG|USER)\b", " ", text, flags=re.I)
    text = re.sub(r"\s+", " ", text)
    return text.strip()

df = pd.read_csv(CSV_PATH, delimiter=delimiter)
df = df.dropna(subset=["tweet", "label"])
df["tweet"] = df["tweet"].astype(str).apply(clean_tweet)

# 3) ──────────────── ENCODE DE ETIQUETAS ───────────────────
labels     = sorted(df["label"].unique())           # p.e. ['anger','joy',...]
label2id   = {lab: idx for idx, lab in enumerate(labels)}
id2label   = {idx: lab for lab, idx in label2id.items()}
df["label"] = df["label"].map(label2id)            

# 4) ──────────────── SPLIT TRAIN / VALID ───────────────────
train_df = df.sample(frac=0.8, random_state=42)
val_df   = df.drop(train_df.index)

train_ds = Dataset.from_pandas(train_df.reset_index(drop=True))
val_ds   = Dataset.from_pandas(val_df.reset_index(drop=True))

# 5) ──────────────── TOKENIZADOR + MODELO ──────────────────
MODEL_NAME = "daveni/twitter-xlm-roberta-emotion-es"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model     = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=len(label2id),
    id2label=id2label,
    label2id=label2id
)

# 6) ──────────────── TOKENIZACIÓN ──────────────────────────
def tokenize(batch):
    return tokenizer(
        batch["tweet"],
        truncation=True,
        padding="max_length",
        max_length=128
    )

train_ds = train_ds.map(tokenize, batched=True)
val_ds   = val_ds.map(tokenize,   batched=True)

cols = ["input_ids", "attention_mask", "label"]
train_ds.set_format(type="torch", columns=cols)
val_ds.set_format(  type="torch", columns=cols)

# 7) ──────────────── ARGUMENTOS DE ENTRENAMIENTO ───────────
training_args = TrainingArguments(
    output_dir            = "./emotion_model",
    evaluation_strategy   = "epoch",
    save_strategy         = "epoch",
    logging_strategy      = "epoch",
    learning_rate         = 2e-5,
    per_device_train_batch_size = 16,
    per_device_eval_batch_size  = 16,
    num_train_epochs      = 3,
    weight_decay          = 0.01,
    load_best_model_at_end= True,
    metric_for_best_model = "accuracy",
)

# 8) ──────────────── MÉTRICAS PERSONALIZADAS ───────────────
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)
    return {
        "accuracy": accuracy_score(labels, preds),
        "f1":       f1_score(labels, preds, average="weighted"),
    }

# 9) ──────────────── TRAINER Y FINE-TUNING ────────────────
trainer = Trainer(
    model           = model,
    args            = training_args,
    train_dataset   = train_ds,
    eval_dataset    = val_ds,
    tokenizer       = tokenizer,
    compute_metrics = compute_metrics,
)

trainer.train()

# 🔽 Guarda la versión fine-tuneada
trainer.save_model("./emotion_model")
tokenizer.save_pretrained("./emotion_model")

print("\n✅ Fine-tuning completado y modelo guardado en ./emotion_model")


tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

...

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.736000,0.556080,0.821452,0.807654
2,0.399900,0.468874,0.852145,0.842183
3,0.245700,0.470041,0.874917,0.865042



✅ Fine-tuning completado y modelo guardado en ./emotion_model
Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7f3220588610>> (for post_run_cell), with arguments args (<ExecutionResult object at 7f322e6e3b10, execution_count=5 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 7f322cbd50d0, raw_cell="# ================================================.." store_history=True silent=False shell_futures=True cell_id=21608e1c-6303-40cf-a441-b7c27fdb5fea> result=None>,),kwargs {}:


TypeError: _WandbInit._pause_backend() takes 1 positional argument but 2 were given